In [21]:
# Imports
import pandas as pd
import os
import sys

# Importing Data

A MultiIndexed DataFrame in pandas is a DataFrame that has multiple levels of indexing on one or both axes (rows and/or columns). This is useful for working with higher-dimensional data in a 2D structure.

Here the indices are:
 - Ticker
 - Date

In [22]:
# Path to the folder containing stock files
folder_path = './data'
sys.path.append(folder_path)

# Prepare a list to hold DataFrames
dataframes = []

# Loop through all files in the directory
for file in os.listdir(folder_path):
    if file.endswith('.us.txt'):
        ticker = file.split('.')[0]  # Extract 'aapl' from 'aapl.us.txt'
        file_path = os.path.join(folder_path, file)

        # Read the CSV
        df = pd.read_csv(file_path, parse_dates=['Date'])

        # Set index to Date
        df.set_index('Date', inplace=True)

        # Add a level to the index for the ticker
        df['Ticker'] = ticker
        df.set_index('Ticker', append=True, inplace=True)

        # Reorder MultiIndex: Ticker -> Date
        df = df.reorder_levels(['Ticker', 'Date'])

        dataframes.append(df)

# Combine all into one MultiIndexed DataFrame
combined_df = pd.concat(dataframes).sort_index()
combined_df.head()

Open     High      Low    Close    Volume  OpenInt
Ticker Date                                                             
aapl   1984-09-07  0.42388  0.42902  0.41874  0.42388  23220030        0
       1984-09-10  0.42388  0.42516  0.41366  0.42134  18022532        0
       1984-09-11  0.42516  0.43668  0.42516  0.42902  42498199        0
       1984-09-12  0.42902  0.43157  0.41618  0.41618  37125801        0
       1984-09-13  0.43927  0.44052  0.43927  0.43927  57822062        0

# Data Cleaning

We first filter the data for past 10 years

In [23]:
end_date = pd.Timestamp('2017-11-10')
start_date = end_date - pd.DateOffset(years=10)

filtered_df = combined_df.loc[
    (combined_df.index.get_level_values('Date') >= start_date) &
    (combined_df.index.get_level_values('Date') <= end_date)
].copy()

filtered_df.head()

Open    High     Low   Close     Volume  OpenInt
Ticker Date                                                          
aapl   2007-11-12  21.130  21.479  19.291  19.691  492362604        0
       2007-11-13  20.615  21.897  19.691  21.765  484373501        0
       2007-11-14  22.733  22.739  20.970  21.274  403585172        0
       2007-11-15  21.280  21.717  20.528  21.040  414487458        0
       2007-11-16  21.193  21.388  20.405  21.309  385660112        0

Now some of the compnies might ha IPO'ed after 2007. Here we identify such companies and use forward fill to fill there data.

In [24]:
# Create the full 10-year date range (business days)
full_dates = pd.date_range(start=start_date, end=end_date, freq='B')

# Function to show missing dates for each ticker
def missing_dates_per_ticker(ticker):
    ticker_dates = filtered_df.loc[ticker].index
    missing = full_dates.difference(ticker_dates)
    return pd.Series(missing, name=ticker)

for ticker in filtered_df.index.get_level_values('Ticker').unique():
    missing_dates = missing_dates_per_ticker(ticker)
    if not missing_dates.empty:
        print(f"Missing dates for {ticker}:")
        print(missing_dates)
        print()  # Print a newline for better readability

Missing dates for aapl:
0    2007-11-22
1    2007-12-25
2    2008-01-01
3    2008-01-21
4    2008-02-18
        ...    
86   2017-02-20
87   2017-04-14
88   2017-05-29
89   2017-07-04
90   2017-09-04
Name: aapl, Length: 91, dtype: datetime64[ns]

Missing dates for amzn:
0    2007-11-22
1    2007-12-25
2    2008-01-01
3    2008-01-21
4    2008-02-18
        ...    
86   2017-02-20
87   2017-04-14
88   2017-05-29
89   2017-07-04
90   2017-09-04
Name: amzn, Length: 91, dtype: datetime64[ns]

Missing dates for googl:
0    2007-11-22
1    2007-12-25
2    2008-01-01
3    2008-01-21
4    2008-02-18
        ...    
86   2017-02-20
87   2017-04-14
88   2017-05-29
89   2017-07-04
90   2017-09-04
Name: googl, Length: 91, dtype: datetime64[ns]

Missing dates for msft:
0    2007-11-22
1    2007-12-25
2    2008-01-01
3    2008-01-21
4    2008-02-18
        ...    
86   2017-02-20
87   2017-04-14
88   2017-05-29
89   2017-07-04
90   2017-09-04
Name: msft, Length: 91, dtype: datetime64[ns]

Missing da

In [33]:
# Reindex each ticker individually
def pad_dates(df):
    df = df.reset_index(level='Ticker', drop=True)  # remove redundant index level
    df = df.reindex(full_dates)  # add missing dates
    df.index.name = 'Date'
    return df.ffill()  # or use .interpolate() if you prefer

# Apply to each ticker
padded_df = filtered_df.groupby(level='Ticker').apply(pad_dates)

# Restore MultiIndex
padded_df = padded_df.reset_index().rename(columns={'level_0': 'Ticker'})
padded_df = padded_df.set_index(['Ticker', 'Date']).sort_index()

padded_df.head()

Open    High     Low   Close       Volume  OpenInt
Ticker Date                                                            
aapl   2007-11-12  21.130  21.479  19.291  19.691  492362604.0      0.0
       2007-11-13  20.615  21.897  19.691  21.765  484373501.0      0.0
       2007-11-14  22.733  22.739  20.970  21.274  403585172.0      0.0
       2007-11-15  21.280  21.717  20.528  21.040  414487458.0      0.0
       2007-11-16  21.193  21.388  20.405  21.309  385660112.0      0.0

# Data Transformation

We calculate and add important informations from data.

They include:
 - **Daily Return**: Percentage change in the closing price from the previous day.
 - **7-Day Moving Average**: Average of the closing price over the past 7 trading days.
 - **30-Day Moving Average**: Average of the closing price over the past 30 trading days.
 - **Rolling Volatility(30d)**: Standard deviation of daily returns over the past 30 trading days.

Here I define a function to add all indicators at once, and use df.apply to apply them.

In [34]:
# Reset index to columns
temp_df = filtered_df.reset_index()

def add_indicators(df):
    df = df.sort_values('Date')
    df['Daily Return'] = df['Close'].pct_change()
    df['7MA'] = df['Close'].rolling(window=7).mean()
    df['30MA'] = df['Close'].rolling(window=30).mean()
    df['Rolling Volatility'] = df['Daily Return'].rolling(window=30).std()
    return df

# Apply on groups of Ticker
enhanced_temp = temp_df.groupby('Ticker').apply(add_indicators)

# Drop the extra Ticker level created by groupby + apply
enhanced_temp.reset_index(level=0, drop=True, inplace=True)

# Set MultiIndex again
enhanced_df = enhanced_temp.set_index(['Ticker', 'Date']).sort_index()

enhanced_df.head()

C:\Users\conta\AppData\Local\Temp\ipykernel_34184\2172446138.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  enhanced_temp = temp_df.groupby('Ticker').apply(add_indicators)


Open    High     Low   Close     Volume  OpenInt  \
Ticker Date                                                             
aapl   2007-11-12  21.130  21.479  19.291  19.691  492362604        0   
       2007-11-13  20.615  21.897  19.691  21.765  484373501        0   
       2007-11-14  22.733  22.739  20.970  21.274  403585172        0   
       2007-11-15  21.280  21.717  20.528  21.040  414487458        0   
       2007-11-16  21.193  21.388  20.405  21.309  385660112        0   

                   Daily Return  7MA  30MA  Rolling Volatility  
Ticker Date                                                     
aapl   2007-11-12           NaN  NaN   NaN                 NaN  
       2007-11-13      0.105327  NaN   NaN                 NaN  
       2007-11-14     -0.022559  NaN   NaN                 NaN  
       2007-11-15     -0.010999  NaN   NaN                 NaN  
       2007-11-16      0.012785  NaN   NaN                 NaN

# Analysis

Q1: Which stock had the highest average return over the 10-year period?

🧠 Approach:

    Use the Daily Return column.

    Group by Ticker, calculate the mean return.

In [35]:
average_returns = enhanced_df['Daily Return'].groupby('Ticker').mean()
highest_avg_return_ticker = average_returns.idxmax()
highest_avg_return_value = average_returns.max()

print(f"Stock with highest average return: {highest_avg_return_ticker} ({highest_avg_return_value:.4%})")

Stock with highest average return: tsla (0.2101%)


Q2: Which stock had the most volatile month, and when?

🧠 Approach:

    Use the Daily Return column.

    Group by Ticker and month (use Date level), compute monthly volatility.

    Find the group with highest standard deviation.

In [36]:
# Reset index to make Date accessible as a column
df_reset = enhanced_df.reset_index()

# Add 'YearMonth' column for monthly grouping
df_reset['YearMonth'] = df_reset['Date'].dt.to_period('M')

# Group by Ticker + YearMonth, calculate std of Daily Return
monthly_volatility = df_reset.groupby(['Ticker', 'YearMonth'])['Daily Return'].std()

# Find the max volatility
most_volatile = monthly_volatility.idxmax()
most_volatile_value = monthly_volatility.max()

print(f"Most volatile month: {most_volatile[0]} in {most_volatile[1]} with std dev of {most_volatile_value:.4%}")

Most volatile month: tsla in 2010-06 with std dev of 28.8362%
